In [1]:
#!/usr/bin/env python
#SBATCH --job-name=SNN
#SBATCH --error=%x.%j.err
#SBATCH --output=%x.%j.out
#SBATCH --mail-user=hzhao@teco.edu
#SBATCH --export=ALL
#SBATCH --time=48:00:00
#SBATCH --partition=sdil
#SBATCH --gres=gpu:1

import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'utils'))
from configuration import *
import torch
import pprint
import BaselineSNN as B
from utils import *

args = parser.parse_args([])
args.projectname = 'test1111'
args.DATASET = 0
args.DEVICE = 'cpu'
args.metric = 'nominal_snn'


train_loader, datainfo = GetDataLoader(args, 'train')
valid_loader, datainfo = GetDataLoader(args, 'valid')
test_loader, datainfo = GetDataLoader(args, 'test')
pprint.pprint(datainfo)

for seed in range(10):
    args.SEED = seed
    args = FormulateArgs(args)

    print(f'Training network on device: {args.DEVICE}.')
    MakeFolder(args)
    
    SetSeed(args.SEED)
    
    setup = f"model_SNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
    print(f'Training setup: {setup}.')

    msglogger = GetMessageLogger(args, setup)
    msglogger.info(f'Training network on device: {args.DEVICE}.')
    msglogger.info(f'Training setup: {setup}.')
    msglogger.info(datainfo)
    

    if os.path.isfile(f'{args.savepath}/{setup}'):
            print(f'{setup} exists, skip this training.')
            msglogger.info('Training was already finished.')
    else:
        topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
        msglogger.info(f'Topology of the network: {topology}.')

        snn = B.SpikingNeuralNetwork(args, topology).to(args.DEVICE)

        lossfunction = B.SNNLoss().to(args.DEVICE)
        optimizer = torch.optim.Adam(snn.GetParam(), lr=args.LR)

        if args.PROGRESSIVE:
            snn, best = train_pnn_progressive(snn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
        else:
            snn, best = train_pnn(pnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

        if best:
            if not os.path.exists(f'{args.savepath}/'):
                os.makedirs(f'{args.savepath}/')
            torch.save(snn, f'{args.savepath}/{setup}')
            msglogger.info('Training if finished.')
        else:
            msglogger.warning('Time out, further training is necessary.')

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 100,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}
Training network on device: cpu.
Training setup: model_SNN_data_acuteinflammation_seed_00.model.
| Epoch:      0 | Train loss: 2.0324e+00 | Valid loss: 2.6567e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 0.4 | Power: 0.00e+00 |
| Epoch:      1 | Train loss: 2.5591e-01 | Valid loss: 8.4449e-01 | Train acc: 0.5286 | Valid acc: 0.9565 | patience:   1 | lr: 0.1 | Epoch time: 0.3 | Power: 0.00e+00 |
| Epoch:      2 | Train loss: 7.6835e-01 | Valid loss: 4.1404e-01 | Train acc: 0.9571 | Valid acc: 0.8696 | patience:   2 | lr: 0.1 | Epoch time: 0.3 | Power: 0.00e+00 |
| Epoch:      3 | Train loss: 3.9333e-01 | Valid loss: 3.8514e-01 | Train acc: 0.8429 | Valid acc: 0.8261 | patience:   3 | lr: 0.1 | Epoch time: 0.3 | Power: 0.00e+00 |
| Epoch:      4 | Train loss: 3.8293e-01 | Valid loss: 3.9119e-01 | Train acc: 0.7429 | Valid 


KeyboardInterrupt



In [8]:
print(f'Training network on device: {args.DEVICE}.')
MakeFolder(args)

train_loader, datainfo = GetDataLoader(args, 'train')
valid_loader, datainfo = GetDataLoader(args, 'valid')
test_loader, datainfo = GetDataLoader(args, 'test')
pprint.pprint(datainfo)

SetSeed(args.SEED)

setup = f"model_SNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
print(f'Training setup: {setup}.')

msglogger = GetMessageLogger(args, setup)
msglogger.info(f'Training network on device: {args.DEVICE}.')
msglogger.info(f'Training setup: {setup}.')
msglogger.info(datainfo)

if os.path.isfile(f'{args.savepath}/{setup}'):
        print(f'{setup} exists, skip this training.')
        msglogger.info('Training was already finished.')
else:
    topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
    msglogger.info(f'Topology of the network: {topology}.')
    
    snn = B.SpikingNeuralNetwork(args, topology).to(args.DEVICE)
    
    lossfunction = B.SNNLoss().to(args.DEVICE)
    optimizer = torch.optim.Adam(snn.GetParam(), lr=args.LR)
    
    if args.PROGRESSIVE:
        snn, best = train_pnn_progressive(snn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
    else:
        snn, best = train_pnn(pnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

    if best:
        if not os.path.exists(f'{args.savepath}/'):
            os.makedirs(f'{args.savepath}/')
        torch.save(snn, f'{args.savepath}/{setup}')
        msglogger.info('Training if finished.')
    else:
        msglogger.warning('Time out, further training is necessary.')

Training network on device: cpu.
{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 100,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}


In [9]:
SetSeed(args.SEED)

setup = f"model_SNN_data_{datainfo['dataname']}_seed_{args.SEED:02d}.model"
print(f'Training setup: {setup}.')

msglogger = GetMessageLogger(args, setup)
msglogger.info(f'Training network on device: {args.DEVICE}.')
msglogger.info(f'Training setup: {setup}.')
msglogger.info(datainfo)

Training setup: model_SNN_data_acuteinflammation_seed_00.model.


In [10]:
if os.path.isfile(f'{args.savepath}/{setup}'):
        print(f'{setup} exists, skip this training.')
        msglogger.info('Training was already finished.')
else:
    topology = [datainfo['N_feature']] + args.hidden + [datainfo['N_class']]
    msglogger.info(f'Topology of the network: {topology}.')
    
    print()

    snn = B.SpikingNeuralNetwork(args, topology).to(args.DEVICE)

In [11]:
lossfunction = B.SNNLoss().to(args.DEVICE)
optimizer = torch.optim.Adam(snn.GetParam(), lr=args.LR)

In [12]:
if args.PROGRESSIVE:
    snn, best = train_pnn_progressive(snn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)
else:
    snn, best = train_pnn(pnn, train_loader, valid_loader, lossfunction, optimizer, args, msglogger, UUID=setup)

if best:
    if not os.path.exists(f'{args.savepath}/'):
        os.makedirs(f'{args.savepath}/')
    torch.save(snn, f'{args.savepath}/{setup}')
    msglogger.info('Training if finished.')
else:
    msglogger.warning('Time out, further training is necessary.')

| Epoch:      0 | Train loss: 2.0324e+00 | Valid loss: 2.6567e-01 | Train acc: 0.5286 | Valid acc: 0.6087 | patience:   0 | lr: 0.1 | Epoch time: 0.3 | Power: 0.00e+00 |
| Epoch:      1 | Train loss: 2.5591e-01 | Valid loss: 8.4449e-01 | Train acc: 0.5286 | Valid acc: 0.9565 | patience:   1 | lr: 0.1 | Epoch time: 0.3 | Power: 0.00e+00 |
| Epoch:      2 | Train loss: 7.6835e-01 | Valid loss: 4.1404e-01 | Train acc: 0.9571 | Valid acc: 0.8696 | patience:   2 | lr: 0.1 | Epoch time: 0.3 | Power: 0.00e+00 |
| Epoch:      3 | Train loss: 3.9333e-01 | Valid loss: 3.8514e-01 | Train acc: 0.8429 | Valid acc: 0.8261 | patience:   3 | lr: 0.1 | Epoch time: 0.3 | Power: 0.00e+00 |
| Epoch:      4 | Train loss: 3.8293e-01 | Valid loss: 3.9119e-01 | Train acc: 0.7429 | Valid acc: 0.7826 | patience:   4 | lr: 0.1 | Epoch time: 0.3 | Power: 0.00e+00 |
| Epoch:      5 | Train loss: 3.9485e-01 | Valid loss: 4.0471e-01 | Train acc: 0.6000 | Valid acc: 0.7391 | patience:   5 | lr: 0.1 | Epoch time: 0.3 


KeyboardInterrupt



In [ ]:
n = 0
c = 0
plt.figure(figsize=(30,5))
plt.plot(X_train[n,c,:]/10, label='input')
plt.plot(spikes[n,c,:].detach().numpy(), label='spike')
plt.plot(memories[n,c,:].detach().numpy(), label='memory')
plt.legend()

In [ ]:
tmac = psnn.TemporalWeightedSum(N_channel, N_class)
tmac

In [ ]:
mactmac = tmac(spikes)
mactmac.shape

In [ ]:
torch.manual_seed(seed)
topology = [N_channel, 3, N_class]
snn = psnn.SpikingNeuralNetwork(topology, beta, threshold)
optimizer = torch.optim.Adam(snn.parameters(), lr=0.01)


In [ ]:
snn.ResetOutput(False, 1)
input = snn.model[0](X_train)
spikes,memories = snn.model[1](input)
n = 0
c = 0
plt.figure(figsize=(30,5))
plt.plot(input[n,c,:].detach().numpy(), label='input')
plt.plot(spikes[n,c,:].detach().numpy(), label='spike')
plt.plot(memories[n,c,:].detach().numpy(), label='memory')
plt.legend()

In [ ]:
torch.manual_seed(seed)
topology = [N_channel, 3, N_class]
snn = psnn.SpikingNeuralNetwork(topology, beta=torch.zeros(1), threshold=torch.ones(1))
optimizer = torch.optim.Adam(snn.parameters(), lr=0.1)
loss_fn = psnn.SNNLoss()

In [ ]:
for epoch in range(1000):
    y_pred_train = snn(X_train)
    loss_train = loss_fn(y_pred_train, y_train)
    acc_train = (y_pred_train.sum(2).argmax(dim=1) == y_train).float().mean()
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    print(f'epoch: {epoch:-8d} | train loss: {loss_train:.5e} | train acc: {acc_train:.4f}')


In [ ]:
idx = torch.randperm(N_train)[:100]

X_visual = X_train[idx,:,:]
y_visual = y_train[idx]

plt.figure(figsize=(30,5))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.plot(X_visual[i,0,:].detach().numpy(), label=f'{y_visual[i]}')
    plt.axis('off')
    plt.legend()

In [1]:
import os
import sys
sys.path.append(os.getcwd())
sys.path.append(os.path.join(os.getcwd(), 'utils'))
from configuration import *
import torch
import pprint
import BaselineSNN as B
from utils import *

args = parser.parse_args([])
args.projectname = 'test1111'
args.DATASET = 0
args.DEVICE = 'cpu'
args.metric = 'nominal_snn'

for ds in range(13):
    args.DATASET = ds
    train_loader, datainfo = GetDataLoader(args, 'train')
    valid_loader, datainfo = GetDataLoader(args, 'valid')
    test_loader, datainfo = GetDataLoader(args, 'test')
    pprint.pprint(datainfo)

{'N_class': 2,
 'N_feature': 6,
 'N_test': 25,
 'N_time': 100,
 'N_train': 70,
 'N_valid': 23,
 'dataname': 'acuteinflammation'}
{'N_class': 3,
 'N_feature': 4,
 'N_test': 79,
 'N_time': 100,
 'N_train': 373,
 'N_valid': 124,
 'dataname': 'balancescale'}
{'N_class': 2,
 'N_feature': 9,
 'N_test': 79,
 'N_time': 100,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'breastcancerwisc'}
{'N_class': 3,
 'N_feature': 21,
 'N_test': 79,
 'N_time': 100,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'cardiotocography3clases'}
{'N_class': 3,
 'N_feature': 8,
 'N_test': 79,
 'N_time': 100,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'energyy1'}
{'N_class': 3,
 'N_feature': 8,
 'N_test': 79,
 'N_time': 100,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'energyy2'}
{'N_class': 3,
 'N_feature': 4,
 'N_test': 31,
 'N_time': 100,
 'N_train': 88,
 'N_valid': 29,
 'dataname': 'iris'}
{'N_class': 2,
 'N_feature': 5,
 'N_test': 79,
 'N_time': 100,
 'N_train': 397,
 'N_valid': 127,
 'dataname': 'mammog